In [11]:
library(RMySQL)

In [12]:
#Enter the values for you database connection
dsn_database = "ad_f21905e045c4fc2"            
            # e.g. "db89c1c13a2014642a38dee00666f9d0c"
dsn_hostname = "us-cdbr-iron-east-03.cleardb.net" # e.g.: "50.23.230.134"
dsn_port = 3306          # e.g. 3307 without quotes
dsn_uid =  "b32f3ab9a16720"            # e.g. "user1"
dsn_pwd = "455fe273"         # e.g. "7dBZ3jWt9xN6$o0JiX!m"

In [13]:
conn = dbConnect(MySQL(), user=dsn_uid, password=dsn_pwd, dbname=dsn_database, host=dsn_hostname)
conn

<MySQLConnection:0,1>

In [14]:
query = "select * from clustertable;";
rs = dbSendQuery(conn, query);
df = fetch(rs, -1);
t<-df

In [9]:
dbDisconnect(conn)

Warning message:
“Closing open result sets”

[1] TRUE

In [9]:
#load libraries 
library(data.table)

ERROR: Error in library(h2o): there is no package called ‘h2o’


In [10]:
setDT(df)

In [1]:
dim(df)

NULL

In [12]:
str(df)

Classes ‘data.table’ and 'data.frame':	8816 obs. of  3 variables:
 $ ASSETTAG : int  4689 4689 4689 4689 4689 4689 4689 4689 4689 4689 ...
 $ TIMESTAMP: int  1491398902 1491398927 1491398979 1491399030 1491399055 1491399082 1491399132 1491399183 1491399234 1491399286 ...
 $ CASETEMP : num  -84.6 -84.7 -85.1 -85.1 -85.5 ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [24]:
install.packages("flexdashboard", type = "source")

Installing package into ‘/gpfs/global_fs01/sym_shared/YPProdSpark/user/s2fc-27d9e54610dc0c-6f2ce6dcb959/R/libs’
(as ‘lib’ is unspecified)
also installing the dependencies ‘backports’, ‘evaluate’, ‘highr’, ‘markdown’, ‘caTools’, ‘rprojroot’, ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘htmltools’, ‘knitr’, ‘htmlwidgets’, ‘rmarkdown’, ‘shiny’

Warning message in install.packages("flexdashboard", type = "source"):
“installation of package ‘httpuv’ had non-zero exit status”Warning message in install.packages("flexdashboard", type = "source"):
“installation of package ‘shiny’ had non-zero exit status”Warning message in install.packages("flexdashboard", type = "source"):
“installation of package ‘flexdashboard’ had non-zero exit status”

In [15]:

# Import Packages
suppressMessages(require(ibmdbR))
suppressMessages(require(lubridate))
suppressMessages(require(dtw))
suppressMessages(require(reshape2))
suppressMessages(require(optparse))
suppressMessages(require(jsonlite))
suppressMessages(require(xts))




# Get time series data from a table
# If an asset is specified, only get the data for that asset
getTimeSeriesData2 <-
  function(
    #dateToProcess,
       #    dataStartDate,
          # assets = NULL,
           allAssetsDF) {
    # Columns in the data table to capture
    # cols_to_capture = 'ASSETTAG,TIMESTAMP,CASETEMP'
    # if (is.null(assets)) {
    #   # If assets parameter is null, return data for all cases after dataStartDate
    #   query <- paste0('SELECT ',cols_to_capture,' FROM ',tableName,' WHERE TIMESTAMP > \'',toString(dataStartDate),' 00:00:00\'ORDER BY "ASSETTAG" ASC, "TIMESTAMP" ASC')
    # } else {
    #   # If assets parameter is not null, return data for that asset after dataStartDate
    #   inStatement = paste0('ASSETTAG IN (\'',paste0(assets,collapse='\',\''),'\')')
    #   query <- paste0('SELECT ',cols_to_capture,' FROM ',tableName,' WHERE ',inStatement,' AND TIMESTAMP > \'',toString(dataStartDate),' 00:00:00\'ORDER BY "ASSETTAG" ASC, "TIMESTAMP" ASC')
    # }
    # # Execute query
    # allAssetsDF <- idaQuery(query)
    
    # Convert CASETEMP to numeric
    allAssetsDF$CASETEMP <- as.numeric(allAssetsDF$CASETEMP)
    # Impute instances when case temp is recorded as exactly -40 (why is this happening?)
    for (i in which(allAssetsDF$CASETEMP == -40)) {
      if (nrow(allAssetsDF) > i & i > 1) {
        if (allAssetsDF[i - 1, "ASSETTAG"] == allAssetsDF[i, "ASSETTAG"] &
            allAssetsDF[i + 1, "ASSETTAG"] == allAssetsDF[i, "ASSETTAG"]) {
          allAssetsDF[i, "CASETEMP"] <-
            mean(c(allAssetsDF[i - 1, "CASETEMP"], allAssetsDF[i + 1, "CASETEMP"]))
        }
      }
    }
    # Convert TIMESTAMP to datetime
    allAssetsDF$TIMESTAMP <-
      round(as.POSIXct(parse_date_time(allAssetsDF$TIMESTAMP, "Ymd HMS"), tz =
                         "UTC"), units = "mins")
    # Round TIMESTAMP to nearest 5 minutes
    allAssetsDF$TIMESTAMP = format(
      strptime("1970-01-01", "%Y-%m-%d", tz = "UTC") +
        round(as.numeric(allAssetsDF$TIMESTAMP) /
                (5 * 60)) * (5 * 60),
      "%Y-%m-%d %H:%M:%S"
    )
    # Make sure the datetime looks like: 2016-01-01 00:00:00
    allAssetsDF$TIMESTAMP = as.POSIXct(as.character(allAssetsDF$TIMESTAMP), tz =
                                         "UTC")
    # Make a new column called DATE from the datetime
    allAssetsDF$DATE = as.Date(allAssetsDF$TIMESTAMP, tz = "UTC")
    # For each DATE, create a new column called DAYS_BACK,
    #   which is 0 for the date that is being processed
    #   and 1 for the day before the date to process, 2 for the next day, etc
    #allAssetsDF$DAYS_BACK = as.integer(round(as.numeric(difftime(as.POSIXlt(dateToProcess,tz="UTC"),as.POSIXlt(allAssetsDF$DATE,tz="UTC"),tz="UTC",units="days"))))
    return(allAssetsDF)
  }







getDecomp2 <-
  function(decompDataFilled,
           freq = 288,
           asset = "UNKNOWN") {
    # Perform the time series decomposition
    # Deal with the rare situation of a time series that starts or ends in NAs
    
    return (ts(decompDataFilled$CASETEMP, freq = freq))
    
    
    
  }




# Given an asset, a data set of values from the STORE*** table, and the pivoted DTW Table,
# calculate the case health scores for that one asset
# date = date to process the scores for
# store = store number
# asset = ASSETTAG
# assetDF = a dataframe containing the data (ASSETTAG, TIMESTAMP, CASETEMP) for this one asset for
#   last few weeks (default = 21 days)
# assetDTWDF = a pivoted assetDF, as described by the function: createDTWtable()
# maxNAprop = if the proportion of missing values in the Date To Process is greater than this value,
#   return NAs for the scores
# hoursInPeriod: how many hours in one period for this case. Default is 24
processAsset2 = function(date,
                         asset,
                         assetDF,
                         maxNAprop = 0.1,
                         hoursInPeriod = 24) {
  
  # Remove any duplicate readings for this asset. Shouldn't be any
  assetDF <- assetDF[!duplicated(assetDF$TIMESTAMP),]
  # In the next few lines, I'm going to try to automatically determine the data frequency
  # Take 2 contingious samples (I used the 2nd and 3rd reading) below
  # Calculate the time difference between them in minutes (sampleIntervalMin)
  # Figure out how many of readings of sampleIntervalMin fit into one period (default = 24 hours)
  # ^ that is the value of "frequency" or "freq" that will be passed to time series decomposition
  # NOTE: There is a small danger here... the timediff between readings 2 and 3 might have a gap
  #     which throws off the calculation of how many readings per day there are.
  sampleIntervalMin = as.numeric(difftime(assetDF$TIMESTAMP[3], assetDF$TIMESTAMP[2], units =
                                            "mins"))
  samplesIn1Period = (hoursInPeriod * 60) / sampleIntervalMin
  # Compile data for decomp into the dataframe: decompData. Order by TIMESTAMP
  decompData = assetDF[order(assetDF$TIMESTAMP), c("TIMESTAMP", "CASETEMP")]
  # There might be missing values in the data. "full" is a sequence from the start of data until the end,
  #    using whatever timediff was present between readings 2 and 3 (arbitrarily selected contingious points)
  # I'm creating a new dataframe that has a row for EVERY point in time that there SHOULD be data.
  #  Then I'm merging the ACTUAL data into that. That way, any missing values in the time series.
  #  That way, we won't have any gaps. They'll be filled with NAs
  full = seq(
    from = min(assetDF$TIMESTAMP),
    to = max(assetDF$TIMESTAMP),
    by = difftime(assetDF$TIMESTAMP[3], assetDF$TIMESTAMP[2], units = "mins")
  )
  decompDataFilled = data.frame(TIMESTAMP = full)
  decompDataFilled = merge(decompDataFilled,
                           decompData[, c("TIMESTAMP", "CASETEMP")],
                           by = "TIMESTAMP",
                           all.x = T)
  decompDataFilled = decompDataFilled[complete.cases(decompDataFilled$TIMESTAMP),]
  # Make sure everything is still ordered by TIMESTAMP
  decompDataFilled = decompDataFilled[order(decompDataFilled$TIMESTAMP), c("TIMESTAMP", "CASETEMP")]
  # Only keep data for days that are on or before the dateToProcess
  #decompDataFilled = decompDataFilled[decompDataFilled$DAYS_BACK >= 0,]
  # Deal with the rare situation of a timeseries that stars or ends with NAs
  while (is.na(decompDataFilled[1, "CASETEMP"])) {
    decompDataFilled = decompDataFilled[2:nrow(decompDataFilled),]
  }
  while (is.na(decompDataFilled[nrow(decompDataFilled), "CASETEMP"])) {
    decompDataFilled = decompDataFilled[1:nrow(decompDataFilled) - 1,]
  }
  # Calculate the time series decomposition scores.
  x <-
    getDecomp2(decompDataFilled, freq = samplesIn1Period, asset = asset)
  
  # Pull out the time series for the date to process
  # and the time series for the historical median.
  # Then, calculate the DTW distance
  # ts1 <- assetDTWDF[,c("0")]
  # ts2 <- assetDTWDF[,c("HIST_MEDIAN")]
  
  # Compile the scores into a new row, which will get added to the health report.
  # assetValues <- data.frame(
  #   ASSET=asset,
  #   STORE=asset,
  #   LASTFULLDAY=date,
  #   TREND1=trend1,
  #   TREND3=trend3,
  #   TREND5=trend5,
  #   RANDOM=volscore,
  #   ANOMALY=anomaly,
  #   NUMNA=propNA
  # )
  return(x)
}






# This is the primary function
# Given a store or list of stores, a start date, and an end date,
#    it will process all the assets in that store (or stores) for the given days
# A list of ASSETTAGs can also be supplied, in which case only those assets will have scores calculated
#    Note: if assetsToProcess is defined, you still need to provide the corresponding store numbers
# host / user / password / dbName: connection details to database
#    If a host is not specified, assume this is running locally within DashDB
# tablePrefix: default is "STORE". We assume the data table for store 919 is: tablePrefix+StoreName = STORE919
# decompLastNDays: How many days worth of data to pull for decomposition. 21 seems to work well.
#    More data means the STL decomp function will see more periods, but it also will take longer to pull data
# getAssetsLastNDays: If no assets are provided, the script gets a list of unique assets in the store table.
#    This function determines how many days back to look for unique asset names
# hoursInPeriod: how many hours in one full period of case temperature data (to capture seasonality). 24 hours is default
getCentroidDTW = function(# stores,
  dateStart = NULL,
  dateEnd = NULL,
  assetsToProcess = NULL,
  decompLastNdays = 21,
  getAssetsLastNdays = 3,
  hoursInPeriod = 24,
  t,
  currentTime) {
  
  
  # Initialize the case health report
  health_rep <- data.frame(
    ASSET = as.character(),
    STORE = as.character(),
    LASTFULLDAY = as.Date(as.character()),
    TREND1 = as.numeric(),
    TREND3 = as.numeric(),
    TREND5 = as.numeric(),
    RANDOM = as.numeric(),
    ANOMALY = as.numeric(),
    NUMNA = as.integer()
  )
  # dateToProcess<- dates
  # dateToProcess = as.Date(dateToProcess, origin = "1970-01-01", tz =
  #                           "UTC")
  
  # allAssetsDF <- t[which(as.POSIXlt(dateEnd+1,origin="1970-01-01", "UTC")-t$TIMESTAMP>0),]
  # allAssetsDF<-t[which(t$TIMESTAMP-as.POSIXlt(dateStart-21,origin="1970-01-01", "UTC")>0),]
  allAssetsDF <- t
  allAssetsDF <-
    getTimeSeriesData2(
      #dateToProcess = dateStartDate,
     # assets = assets,
     # dataStartDate = dateStartDate,
      allAssetsDF
    )
  
  distMatrix<- data.frame()
  score<-data.frame()
  # Pivot the asset data as described in the createDTWtable.
  # The primary key for each row is an ASSETTAG and a time of day (examples: 00:00, 00:05, etc)
  # The column "0" contains the time series for that ASSETTAG at the time of day on the date to process
  # The column "1" contains the time series for that ASSETTAG at the time of day for N-1 days
  # The column "2" contains the time series for that ASSETTAG at the time of day for N-2 days
  # allDtwDF <- createDTWtable(allAssetsDF,dtwUseLastNDays=decompLastNdays)
  
  dfTimeSeries <- data.frame()
  # Now that the data tables have been all setup and we have a list of assets...
  # calculate the case health scores for each of the assets in this store
  #for (asset in assets) {
  assetList <- unique(t$ASSETTAG)
  i <- 1
  
  
 distMatrix<- matrix(1:100,10,10)
  for (i in 1:10)
  {
    asset1 <- assetList[i]
    for (j in 1:10)
    {
      if (i != j)
      {
        asset2 <- assetList[j]
        
        # Just get the subset of the store data for this asset
        assetDF1 <-
          allAssetsDF[allAssetsDF$ASSETTAG == asset1, c("TIMESTAMP", "CASETEMP")]
        tsAsset1 <-
          processAsset2(
            date = dateToProcess,
            asset1,
            assetDF1,
            maxNAprop = 0.5,
            hoursInPeriod = hoursInPeriod
          )
        assetDF2 <-
          allAssetsDF[allAssetsDF$ASSETTAG == asset2, c("TIMESTAMP", "CASETEMP")]
        tsAsset2 <-
          processAsset2(
            date = dateToProcess,
            asset2,
            assetDF2,
            maxNAprop = 0.5,
            hoursInPeriod = hoursInPeriod
          )
        dfTimeSeries <- t(as.data.frame(tsAsset1))
        dfTimeSeries2 <- t(as.data.frame(tsAsset2))
        dfTimeSeries[,which(is.na(dfTimeSeries))]<- mean(assetDF1$CASETEMP)
        dfTimeSeries2[,which(is.na(dfTimeSeries2))]<- mean(assetDF2$CASETEMP)
        if(ncol(dfTimeSeries)<ncol(dfTimeSeries2))
        {dfTimeSeries3 <-
          rbind(dfTimeSeries, dfTimeSeries2[, 1:ncol(dfTimeSeries)])}
        else
        {dfTimeSeries3 <-
          rbind(dfTimeSeries[, 1:ncol(dfTimeSeries2)], dfTimeSeries2)}
        distMatrix[i, j] <- dist(dfTimeSeries3, method = 'DTW')
        
      }
    }
    
  }
  
  return(distMatrix)
}

getClusterDTW = function(# stores,
  dateStart = NULL,
  dateEnd = NULL,
  assetsToProcess = NULL,
  decompLastNdays = 21,
  getAssetsLastNdays = 3,
  hoursInPeriod = 24,
  t,
  currentTime) {
  # Determine what day it is today, and when the last full day's worth of data is (yesterday)
  
  # currentDate = as.Date(currentTime,origin="1970-01-01",tz="UTC")
  # lastFullDay = as.Date(currentDate - 1,origin="1970-01-01",tz="UTC")
  #
  # # If you didn't specify a day to start on, assume you want yesterday's scores
  # dateStart = ifelse(is.null(dateStart),lastFullDay,dateStart)
  # #dateStart<-lastFullDay
  # dateStart = as.Date(dateStart,origin="1970-01-01",tz="UTC")
  # #dateEnd="2015-06-01T00:00:18Z UTC"
  # # If you didn't specify a day to end on, assume you only want dateStart's scores
  # dateEnd = lastFullDay
  # dateEnd = as.Date(dateEnd,origin="1970-01-01",tz="UTC")
  # Create a sequence of dates between dateStart and dateEnd
  datesToProcess = as.Date(seq.Date(dateStart, dateEnd, by = "days"), tz =
                             "UTC")
  
  dfTimeSeries3<-data.frame()
  # Initialize the case health report
  health_rep <- data.frame(
    ASSET = as.character(),
    STORE = as.character(),
    LASTFULLDAY = as.Date(as.character()),
    TREND1 = as.numeric(),
    TREND3 = as.numeric(),
    TREND5 = as.numeric(),
    RANDOM = as.numeric(),
    ANOMALY = as.numeric(),
    NUMNA = as.integer()
  )
  
  dateToProcess = as.Date(dateToProcess, origin = "1970-01-01", tz =
                            "UTC")
  
  # allAssetsDF <- t[which(as.POSIXlt(dateEnd+1,origin="1970-01-01", "UTC")-t$TIMESTAMP>0),]
  # allAssetsDF<-t[which(t$TIMESTAMP-as.POSIXlt(dateStart-21,origin="1970-01-01", "UTC")>0),]
  allAssetsDF <- t
  allAssetsDF <-
    getTimeSeriesData2(
      #dateToProcess = dateToProcess,
      #assets = assets,
      #dataStartDate = dateStartDate,
      allAssetsDF
    )
  # Pivot the asset data as described in the createDTWtable.
  # The primary key for each row is an ASSETTAG and a time of day (examples: 00:00, 00:05, etc)
  # The column "0" contains the time series for that ASSETTAG at the time of day on the date to process
  # The column "1" contains the time series for that ASSETTAG at the time of day for N-1 days
  # The column "2" contains the time series for that ASSETTAG at the time of day for N-2 days
  # allDtwDF <- createDTWtable(allAssetsDF,dtwUseLastNDays=decompLastNdays)
  
  dfTimeSeries3 <- data.frame()
  # Now that the data tables have been all setup and we have a list of assets...
  # calculate the case health scores for each of the assets in this store
  #for (asset in assets) {
  assetList <- unique(t$ASSETTAG)
  i <- 1
  for (i in 1:10)
  {
    asset <- assetList[i]
    
    
    # Just get the subset of the store data for this asset
    assetDF <-
      allAssetsDF[allAssetsDF$ASSETTAG == asset, c("TIMESTAMP", "CASETEMP")]
    tsAsset <-
      processAsset2(
        date = dateToProcess,
        asset,
        assetDF,
        maxNAprop = 0.5,
        hoursInPeriod = hoursInPeriod
      )
    
    dfTimeSeries <- t(as.data.frame(tsAsset))
    
  
    
    if((i==1))
    {dfTimeSeries3 <- dfTimeSeries
    } else{
      if(ncol(dfTimeSeries)<ncol(dfTimeSeries3))
      {dfTimeSeries3 <-
        rbind(dfTimeSeries3, dfTimeSeries[, 1:ncol(dfTimeSeries)])}
      else
      {dfTimeSeries3 <-
        rbind(dfTimeSeries3,  dfTimeSeries[, 1:ncol(dfTimeSeries3)])}
      
  
      }
    
  #   else
  #   {dfTimeSeries3 <-
  #     rbind(dfTimeSeries3, dfTimeSeries[, 1:ncol(dfTimeSeries)])}
  #   
  }
  return(dfTimeSeries3)
  # distMatrix_all <- dist(dfTimeSeries3, method = 'DTW')
  # hc <- hclust(distMatrix_all, method = 'average')
  # plot(hc, labels = observedLabels, main = '')
  # 
}

writeHealthRepToFile = function(health_rep,
                                filenamePrefix = "health_report_001_",
                                fileNameSuffix = "_scaledVolScore.csv",
                                dateStart = "unknownStartDate",
                                dateEnd = "unknownEndDate")
{
  fileNameToWrite = paste0(filenamePrefix, dateStart, "_to_", dateEnd, fileNameSuffix)
  warning(paste0("Current Directory: ", getwd()))
  warning(paste0("Writing to file: ", fileNameToWrite))
  write.table(
    health_rep,
    fileNameToWrite,
    row.names = F,
    quote = F,
    sep = ","
  )
}


# Given a health report, write the table to the database
# health_rep is the health report data frame
writeHealthRepToDB = function(health_rep,
                              host = NULL,
                              user = "bluadmin",
                              password = "ZmUzYzFhOWMzODI3",
                              dbName = "BLUDB",
                              tempTable = "CASEHEALTH_TMP",
                              caseHealthTable = "CASEHEALTH") {
  health_rep$ASSET = as.character(health_rep$ASSET)
  health_rep = health_rep[!duplicated(health_rep),]
  
  warning(
    paste0(
      "Attempting to write entire health report to casehealth temp table: ",
      tempTable
    )
  )
  # Save the entire health report dataframe to CASEHEALTH_TMP in dashDB (overwrite whatever is there)
  idf = tryCatch({
    idf <- as.ida.data.frame(health_rep, tempTable, clear.existing = T)
  }, warning = function(w) {
    warning("Warning while trying to upload Case Health Report as CASEHEALTH_TMP")
    warning(w)
  }, error = function(e) {
    warning("Error while trying to upload Case Health Report as CASEHEALTH_TMP")
    warning(e)
  })
  
  # Go through each row in the new health report.
  # First, try sqlSave. That may fail if the key (ASSET, LASTFULLDAY) already exists
  # Next, try sqlUpdate. Should overwrite the existing value if the primary key already exists
  warning(
    paste0(
      "Attempting to write individual rows of data to Case Health table: ",
      caseHealthTable
    )
  )
  for (i in 1:nrow(health_rep)) {
    result = tryCatch({
      sqlSave(
        channel = con,
        dat = health_rep[i,],
        tablename = caseHealthTable,
        verbose = F,
        append = T,
        rownames = F,
        colnames = F
      )
    }, warning = function(w) {
      warning("Warning while trying to sqlSave row to table")
      warning(w)
    }, error = function(e) {
      warning("Error while trying to sqlSave row to table. Trying sqlUpdate instead")
      warning(e)
      tryCatch({
        sqlUpdate(
          channel = con,
          dat = health_rep[i,],
          tablename = caseHealthTable,
          verbose = F,
          index = c("ASSET", "LASTFULLDAY")
        )
      }, warning = function(w) {
        warning("Warning while trying to sqlUpdate row to table")
        warning(w)
      }, error = function(e) {
        warning("Error while trying to sqlUpdate row to table")
        warning(e)
      })
    })
  }
  
  return(health_rep)
  
}










################### RUN THE CASE HEALTH  ###############################


t <- read.csv("21days - 10 Sensors.csv")
t$TIMESTAMP <- as.POSIXlt(t$TIMESTAMP, origin = "1970-01-01", "UTC")
currentDate = as.Date(max(t$TIMESTAMP, na.rm = TRUE),
                      origin = "1970-01-01",
                      tz = "UTC")
currentTime = as.POSIXlt(max(t$TIMESTAMP, na.rm = TRUE), origin = "1970-01-01", "UTC")
dateStart <-
  as.Date(currentDate - 21, origin = "1970-01-01", tz = "UTC")
dateEnd <-
  as.Date(currentDate , origin = "1970-01-01", tz = "UTC")
dateToProcess<- dateStart





health_rep_centroid = getCentroidDTW(
  # stores=stores,
  dateStart = dateStart,
  dateEnd = dateEnd,
  assetsToProcess = NULL,
  decompLastNdays = 21,
  getAssetsLastNdays = 3,
  hoursInPeriod = 24,
  t,
  currentTime
)
write.csv(file = "health_rep_centroid.csv", health_rep_centroid)

health_rep_cluster = getClusterDTW(
  # stores=stores,
  dateStart = dateStart,
  dateEnd = dateEnd,
  assetsToProcess = NULL,
  decompLastNdays = 21,
  getAssetsLastNdays = 3,
  hoursInPeriod = 24, 
  t,
  currentTime
) 


Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lubridate’”Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘dtw’”Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘optparse’”Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘xts’”Warning message in file(file, "rt"):
“cannot open file '21days - 10 Sensors.csv': No such file or directory”

ERROR: Error in file(file, "rt"): cannot open the connection


In [17]:

install.packages(lubridate)
install.packages(dtw)
install.packages(reshape2)
install.packages(optparse)
install.packages(jsonlite)
install.packages(xts)


ERROR: Error in install.packages(lubridate): object 'lubridate' not found
